In [51]:
import pandas as pd
import pickle
import numpy as np
import jellyfish

# Read in Courts Mapper
courts_map = pd.read_pickle('courts.pickle')

# Read in Race Mapper File
race_map = pd.read_pickle('race_map.pickle')

# Read in District Data -- takes a couple minutes
district = pd.read_pickle('district_misdemeanor_felony.pickle')

In [75]:
district.head()

,HearingDate,HearingResult,HearingPlea,HearingContinuanceCode,HearingType,HearingCourtroom,fips,FiledDate,Locality,Status,...,Costs,FineCostsDue,FineCostsPaid,FineCostsPaidDate,VASAP,FineCostsPastDue,person_id,PublicDefender,ChargeAmended,Court
2,2010-12-15,Finalized,NaN,NaN,NaN,NaN,91,2010-11-15,COMMONWEALTH OF VA,Released On Summons,...,30.0,NaN,t,2010-12-15,NaN,NaN,212030000000205,False,False,Highland General District Court
22,2010-12-15,Finalized,NaN,NaN,NaN,NaN,91,2010-11-09,COMMONWEALTH OF VA,Other,...,116.0,NaN,t,2011-03-04,NaN,NaN,179180000000606,False,False,Highland General District Court
27,2010-12-15,Finalized,NaN,NaN,Arraignment,NaN,91,2010-12-01,COMMONWEALTH OF VA,Released On Summons,...,166.0,NaN,t,2011-01-06,NaN,NaN,285120000000210,False,False,Highland General District Court
60,2010-12-01,Finalized,NaN,NaN,NaN,NaN,91,2010-07-14,COMMONWEALTH OF VA,Released On Summons,...,30.0,NaN,t,2010-11-17,NaN,NaN,179000000000133,False,True,Highland General District Court
61,2010-12-01,Finalized,NaN,NaN,Review Progress,NaN,91,2010-07-15,COMMONWEALTH OF VA,Released On Summons,...,81.0,NaN,t,2010-11-01,NaN,NaN,123020000000032,False,False,Highland General District Court


In [53]:
# Feature Creation

# Create Public Defender Column
district['PublicDefender'] = district.DefenseAttorney.isin(['Public Defender', 
                                                               'PD', 
                                                               'PUB DEF', 
                                                               'P DEF', 
                                                               'PUBLIC DEFENDER(TSR)', 
                                                               'P/D', 
                                                               'PUBLIC DEFENDER 703-934-5600', 
                                                               'PUBLIC DEFENDER  703-934-5600',
                                                               'PUBLIC DEFENDER 934-5600',
                                                               'PUBLIC DEFENDER (TSR)',
                                                               'PUBLIC DEF',
                                                               'PDEF']+
                                                                [i for i in dict(district.DefenseAttorney.value_counts()) if 'PD' in i]
                                                           )

# Create Column indicating if a charge was amended
district['ChargeAmended'] = district.AmendedCharge.notnull()

# Add in Courts Data from FIPS
district['Court'] = district['fips'].map(courts_map)

# TODO use person column to create "Previous court record in va"

In [78]:
# Clean Data

# Clean Mappings for Race variable
district.Race = district.Race.map(race_map)

# Give Value for NULLs Charge Class
district.Class = district.Class.fillna('None')

# Clean Charge Code Section - Drop NULLs (6 records)
district = district[district['CodeSection'].notnull()]

# Give NUll Sentence Time 0 values - as no sentence would equate to 0
district['SentenceTime'] = district['SentenceTime'].fillna(0).astype(int)

# Drop Records where FinalDisposition is NULL (89854 records)
district = district[district.FinalDisposition.notnull()]

# Drop Records where Complainant is NULL (1 record)
district = district[district.Complainant.notnull()]

In [64]:
# Subsect Data Frame for only Response and Predictor Variables
usecols = ['FinalDisposition', 'SentenceTime', 'Court', 'Complainant', 'PublicDefender', 'Gender', 'Race', 'CaseType', 'Class', 'CodeSection', 'ChargeAmended']

In [56]:
# Write Data for Model
# district[usecols].to_pickle('model_data.pickle')

# Data Exploration

In [ ]:
#TODO